# Step 1 - Scraping


## NASA Mars News

In [1]:
# Dependencies
from bs4 import BeautifulSoup
from splinter import Browser
import requests
import pymongo
import pandas as pd 
import time
import datetime

In [2]:
# Define executable path
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [43]:
# Create empty dictionary 
mars_info_dict=dict()

In [3]:
#define URL
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

In [4]:
html = browser.html
news_soup = BeautifulSoup(html, 'html.parser')

In [5]:
slide_elem = news_soup.select_one('ul.item_list li.slide')

In [6]:
slide_elem.find("div", class_='content_title')

<div class="content_title"><a href="/news/8430/nasas-insight-detects-first-likely-quake-on-mars/" target="_self">NASA's InSight Detects First Likely 'Quake' on Mars</a></div>

In [7]:
news_title = slide_elem.find("div", class_='content_title').get_text()
news_title

"NASA's InSight Detects First Likely 'Quake' on Mars"

In [8]:
# Paragraph teaser
news_para = slide_elem.find('div', class_="article_teaser_body").get_text()
news_para

'While their causes are still unknown, one of three shaking events looks a lot like the quakes detected on the Moon by the Apollo missions.'

In [46]:
from pprint import pprint
mars_info_dict["Mars_news_title"] = news_title
mars_info_dict["Mars_news_body"] = news_para
pprint(mars_info_dict)

{'Mars_news_body': 'While their causes are still unknown, one of three shaking '
                   'events looks a lot like the quakes detected on the Moon by '
                   'the Apollo missions.',
 'Mars_news_title': "NASA's InSight Detects First Likely 'Quake' on Mars"}


In [9]:
# Scrape the NASA Mars News Site and collect the latest News Title and Paragraph Text. 
# Assign the text to variables that you can reference later.

## JPL Mars Space Images - Featured Image

In [10]:
# URL 2 - https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars
url2 = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url2)

In [11]:
# Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called featured_image_url.
# Make sure to find the image url to the full size .jpg image.
# Make sure to save a complete url string for this image.

In [12]:
full_image_elem = browser.find_by_id('full_image')
full_image_elem.click()

In [13]:
browser.is_element_present_by_text('more info', wait_time=1)
more_info_elem = browser.find_link_by_partial_text('more info')
more_info_elem.click()

In [14]:
html2 = browser.html
img_soup = BeautifulSoup(html2, 'html.parser')

In [15]:
#print (image_soup)

In [16]:
img_url_rel = img_soup.select_one('figure.lede a img').get("src")
img_url_rel

'/spaceimages/images/largesize/PIA17474_hires.jpg'

In [17]:
img_url_final = f'https://www.jpl.nasa.gov{img_url_rel}'

In [18]:
img_url_final

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA17474_hires.jpg'

In [48]:
#append into dict

mars_info_dict["Mars_featured_image_url"] = img_url_final
mars_info_dict

{'Mars_news_title': "NASA's InSight Detects First Likely 'Quake' on Mars",
 'Mars_news_body': 'While their causes are still unknown, one of three shaking events looks a lot like the quakes detected on the Moon by the Apollo missions.',
 'Mars_featured_image_url': 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA17474_hires.jpg'}

## Mars Weather

In [20]:
urltwt = 'https://twitter.com/marswxreport?lang=en'
browser.visit(urltwt)
htmltwt = browser.html
souptwt = BeautifulSoup(htmltwt, 'html.parser')

In [ ]:
# Scrape the latest Mars weather tweet from the page. 
# Save the tweet text for the weather report as a variable called mars_weather.

In [21]:
mars_weather = souptwt.find('p', class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text").text
mars_weather

'InSight sol 143 (2019-04-22) low -98.0ºC (-144.3ºF) high -19.8ºC (-3.7ºF)\nwinds from the SW at 4.5 m/s (10.1 mph) gusting to 12.1 m/s (27.2 mph)\npressure at 7.40 hPapic.twitter.com/EOazNkJqjD'

In [49]:
mars_info_dict["Mars_Twitter_Weather"] = mars_weather

## Mars Facts

In [ ]:
# Visit the Mars Facts webpage here and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
# Use Pandas to convert the data to a HTML table string.

In [32]:
url3 = "http://space-facts.com/mars/"
df_marsfacts_all = pd.read_html(url3)
df_marsfacts = df_marsfacts_all[0]
# df_marsfacts

In [33]:
#change column names
df_marsfacts.columns = ['Mars Facts', 'Values']


In [31]:
df_marsfacts

,Mars Facts,Values
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [36]:
df_marsfacts.to_html("mars_facts.html", index=False)
#set index for better retrieval
df_marsfacts.set_index("Mars Facts")

,Values
Mars Facts,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [50]:
mars_facts_html = df_marsfacts.to_html(classes="mars_facts table table-striped")
mars_info_dict["Mars_facts_table"] = mars_facts_html

## Mars Hemispheres

In [ ]:
# Visit the USGS Astrogeology site here to obtain high resolution images for each of Mar's hemispheres.
# You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.
# Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name. Use a Python dictionary to store the data using the keys img_url and title.
# Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary for each hemisphere.

In [38]:
url4 =  "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url4)
time.sleep(10)
html4 = browser.html
soup4 = BeautifulSoup(html4, "html.parser")

In [39]:
#parse soup object for images
class_collap_results = soup4.find('div', class_="collapsible results")
hemis_items = class_collap_results.find_all('div',class_='item')

In [40]:
# click through the images

hemis_img_urls_list=list()
img_urls_list = list()
title_list = list()
for h in hemis_items:
    #save title
    h_title = h.h3.text
    title_list.append(h_title)
    
    # find the href link.
    h_href  = "https://astrogeology.usgs.gov" + h.find('a',class_='itemLink product-item')['href']
    
    #print(h_title,h_href)
    
    #browse the link from each page
    browser.visit(h_href)
    time.sleep(5)
    #Retrieve the  image links and store in a list. 
    html4   = browser.html
    soup_img = BeautifulSoup(html4, 'html.parser')
    h_img_url = soup_img.find('div', class_='downloads').find('li').a['href']
    print("h_img_url" + h_img_url)
    img_urls_list.append(h_img_url)
    
    # create a dictionary with  each image and title and append to a list. 
    hemispheres_dict = dict()
    hemispheres_dict['title'] = h_title
    hemispheres_dict['img_url'] = h_img_url
    
    hemis_img_urls_list.append(hemispheres_dict)
    
print(hemis_img_urls_list)
print(title_list)
print(img_urls_list)

h_img_urlhttp://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg
h_img_urlhttp://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg
h_img_urlhttp://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg
h_img_urlhttp://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg
[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/

In [51]:
#add to the dictionary
mars_info_dict["Hemisphere_image_urls"] = hemis_img_urls_list

In [52]:
cur_datetime = datetime.datetime.utcnow()
mars_info_dict["Date_time"] = cur_datetime

In [53]:
pprint(mars_info_dict)

{'Date_time': datetime.datetime(2019, 4, 24, 3, 27, 4, 130215),
 'Hemisphere_image_urls': [{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
                            'title': 'Cerberus Hemisphere Enhanced'},
                           {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
                            'title': 'Schiaparelli Hemisphere Enhanced'},
                           {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
                            'title': 'Syrtis Major Hemisphere Enhanced'},
                           {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
                            'title': 'Valles Marineris Hemisphere Enhanced'}],
 'Mars_Twitter_Weather': 'InSight sol 143 (2019-04-22) low -98.0ºC (-144.3ºF) '
       

## Step 2 - MongoDB and Flask Application